## Generative AI - Inference(via LangChain)

In [2]:
from os import getenv
from dotenv import load_dotenv
from langchain_community.llms.oci_generative_ai import OCIGenAI

load_dotenv()

COMPARTMENT_ID = getenv("COMPARTMENT_ID")
SERVICE_ENDPOINT = getenv("GEN_AI_INFERENCE_ENDPOINT") # https://inference.generativeai.us-chicago-1.oci.oraclecloud.com

### Using API Key
#### cohere api key

In [5]:
llm = OCIGenAI(
    model_id="cohere.command",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id=COMPARTMENT_ID,
    auth_type="SECURITY_TOKEN",
    auth_profile="yuxing.shao@zdh.co.jp",  # replace with your profile name
    model_kwargs={"temperature": 0.7, "top_p": 0.75, "max_tokens": 200},
)

ValidationError: 1 validation error for OCIGenAI
__root__
  Could not authenticate with OCI client. Please check if ~/.oci/config exists. If INSTANCE_PRINCIPLE or RESOURCE_PRINCIPLE is used, Please check the specified auth_profile and auth_type are valid. (type=value_error)

In [13]:
llm = OCIGenAI(
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command",
    auth_type="API_KEY",
    auth_profile="CHICAGO",
    model_kwargs={
        'max_tokens': 100
    }
)

response = llm.invoke("Tell me a joke", temperature=0.7)

print(response)

ValidationError: 1 validation error for OCIGenAI
__root__
  Could not authenticate with OCI client. Please check if ~/.oci/config exists. If INSTANCE_PRINCIPLE or RESOURCE_PRINCIPLE is used, Please check the specified auth_profile and auth_type are valid. (type=value_error)

In [6]:
llm = OCIGenAI(
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command-light",
    auth_profile="yuxing.shao@zdh.co.jp"
)

response = llm.invoke("Tell me a joke", temperature=0.7)

print(response)

ValidationError: 1 validation error for OCIGenAI
__root__
  Could not authenticate with OCI client. Please check if ~/.oci/config exists. If INSTANCE_PRINCIPLE or RESOURCE_PRINCIPLE is used, Please check the specified auth_profile and auth_type are valid. (type=value_error)

In [11]:
llm = OCIGenAI(
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="meta.llama-2-70b-chat",
    auth_profile="CHICAGO"
)

response = llm.invoke("Tell me a joke", temperature=0.7)

print(response)

ValidationError: 1 validation error for OCIGenAI
__root__
  Could not authenticate with OCI client. Please check if ~/.oci/config exists. If INSTANCE_PRINCIPLE or RESOURCE_PRINCIPLE is used, Please check the specified auth_profile and auth_type are valid. (type=value_error)

### Using Instance Principal

In [1]:
llm = OCIGenAI(
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command",
    # ... omit ...
    auth_type="INSTANCE_PRINCIPAL"
)

response = llm.invoke("Tell me a joke", temperature=0.7)
print(response)

NameError: name 'OCIGenAI' is not defined

### Using Session Token

In [ ]:
llm = OCIGenAI(
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command",
    auth_type="SECURITY_TOKEN",
    auth_profile="langchain"
)

response = llm.invoke("Tell me a joke", temperature=0.7)
print(response)

## Generative AI - Embeddings

In [ ]:
from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings

In [ ]:
embeddings = OCIGenAIEmbeddings(
    model_id="cohere.embed-english-v3.0",
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    auth_type="INSTANCE_PRINCIPAL"
)

response = embeddings.embed_query(text="This is a query in English.")
print(response)

In [ ]:
response = embeddings.embed_documents(
    texts=[
        "This is a first document in English",
        "This is a second document in English"
    ]
)

print(response)

## Prompt Template

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

llm_cohere = OCIGenAI(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command"
)

prompt = PromptTemplate(input_variables=["query"], template="{query}")
llm_chain = LLMChain(llm=llm_cohere, prompt=prompt)

response = llm_chain.invoke("what is the capital of france?")

print(response)

## Retrieval-Augmented Generation(FAISS)

In [ ]:
%pip install faiss-cpu

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS

In [ ]:
import pandas as pd
from langchain_core.documents import Document

In [ ]:
df = pd.read_json('../data/oci-services.jsonl', orient='records', lines=True)
df.head()

In [ ]:
documents: list[Document]= []
for index, row in df.iterrows():
    documents.append(Document(
        page_content=row['text'],
        metadata={
            'title': row['title'],
            'category': row['category'],
            'url': row['url']
        }
    ))


In [ ]:
from langchain.globals import set_debug

set_debug(True)

embeddings = OCIGenAIEmbeddings(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=SERVICE_ENDPOINT,
    model_id="cohere.embed-english-v3.0",
    compartment_id=COMPARTMENT_ID
)

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template=template)
llm = OCIGenAI(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=SERVICE_ENDPOINT,
    model_id="cohere.command",
    compartment_id=COMPARTMENT_ID,
    model_kwargs={
        'max_tokens': 100
    }
)

chain = (
    { "context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(chain.invoke("What is region?"))